In [1]:
import sqlite3, os, datetime as dt
import pandas as pd
import plotly.express as px
from ipywidgets import VBox, HBox, Dropdown, IntSlider, Button, Output, HTML

In [2]:
DB_PATH = "shop.db"
if os.path.exists(DB_PATH):
    os.remove(DB_PATH)

con = sqlite3.connect(DB_PATH, check_same_thread=False)
cur = con.cursor()

# Schema
cur.executescript("""
CREATE TABLE products (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    price REAL NOT NULL,
    stock INTEGER NOT NULL
);

CREATE TABLE orders (
    id INTEGER PRIMARY KEY,
    product_id INTEGER NOT NULL,
    qty INTEGER NOT NULL,
    total REAL NOT NULL,
    created_at TEXT NOT NULL,
    FOREIGN KEY(product_id) REFERENCES products(id)
);
""")

# Dummy seed data
products = [
    ("USB-C Cable", 9.99, 100),
    ("Wireless Mouse", 24.90, 50),
    ("Keyboard", 39.00, 40),
    ("27in Monitor", 189.00, 15),
    ("Laptop Stand", 29.90, 60),
]
cur.executemany("INSERT INTO products(name, price, stock) VALUES (?, ?, ?);", products)
con.commit()

In [3]:
def df(sql, params=()):
    return pd.read_sql(sql, con, params=params)

def place_order(product_id: int, qty: int):
    # Check stock and price
    p = df("SELECT id, price, stock FROM products WHERE id = ?;", (product_id,))
    if p.empty:
        raise ValueError("Product not found.")
    price, stock = float(p.price.iloc[0]), int(p.stock.iloc[0])
    if qty <= 0:
        raise ValueError("Quantity must be > 0.")
    if qty > stock:
        raise ValueError(f"Insufficient stock. Available: {stock}")

    total = round(price * qty, 2)
    cur.execute(
        "INSERT INTO orders(product_id, qty, total, created_at) VALUES (?,?,?,?);",
        (product_id, qty, total, dt.datetime.now(dt.UTC).isoformat())
    )
    cur.execute(
        "UPDATE products SET stock = stock - ? WHERE id = ?;",
        (qty, product_id)
    )
    con.commit()
    return total

def sales_summary():
    orders = df("SELECT * FROM orders ORDER BY created_at;")
    products = df("SELECT * FROM products ORDER BY id;")
    revenue = float(orders.total.sum()) if not orders.empty else 0.0
    return orders, products, revenue

In [4]:
import asyncio
from ipywidgets import Checkbox

# (keep your existing helpers/DB code above...)

# Build product choices once per refresh (unchanged)
def product_options():
    p = df("SELECT id, name, price, stock FROM products ORDER BY id;")
    return [(f"{row['name']} (${row['price']:.2f}) — stock: {row['stock']}", int(row['id'])) for _, row in p.iterrows()]

prod_dd = Dropdown(description="Product:", options=product_options())
qty_sl = IntSlider(description="Qty", min=1, max=10, value=1)
order_btn = Button(description="Place order", button_style="success")
msg_out = Output()

# --- NEW controls ---
refresh_btn = Button(description="Refresh", tooltip="Re-query DB and redraw")
auto_cb = Checkbox(value=False, description="Auto-refresh (2s)")
last_ref_html = HTML()

# Dashboard outputs (unchanged)
kpi_html = HTML()
orders_out = Output()
stock_out = Output()
charts_out = Output()

def refresh_dashboard():
    # Refresh dropdown labels to show updated stock
    prod_dd.options = product_options()

    orders, products, revenue = sales_summary()
    kpi_html.value = (
        f"<h4 style='margin:4px 0'>KPI</h4>"
        f"<div>Total orders: <b>{len(orders)}</b> &nbsp; | &nbsp; "
        f"Total revenue: <b>${revenue:,.2f}</b></div>"
    )

    with orders_out:
        orders_out.clear_output()
        display(orders if not orders.empty else pd.DataFrame(columns=["id","product_id","qty","total","created_at"]))

    with stock_out:
        stock_out.clear_output()
        display(products)

    with charts_out:
        charts_out.clear_output()
        if not products.empty:
            fig_stock = px.bar(products, x="name", y="stock", title="Stock by product")
            fig_stock.show()
        if not orders.empty:
            by_time = orders.copy()
            by_time["created_at"] = pd.to_datetime(by_time["created_at"])
            by_time = by_time.groupby(pd.Grouper(key="created_at", freq="1min")).agg({"total":"sum"}).reset_index()
            fig_sales = px.line(by_time, x="created_at", y="total", title="Revenue over time (sum per minute)")
            fig_sales.show()

# --- OPTIONAL: make errors visible in the message box ---
@msg_out.capture(clear_output=True)  # shows prints/exceptions in msg_out
def on_place_order(_):
    try:
        total = place_order(prod_dd.value, qty_sl.value)
        print(f"✅ Order placed. Charged ${total:.2f}.")
    except Exception as e:
        print(f"❌ {e}")
    refresh_dashboard()
order_btn.on_click(on_place_order)

# --- NEW: manual refresh handler ---
def on_refresh(_=None):
    refresh_dashboard()
    last_ref_html.value = f"<small>Last refresh: {pd.Timestamp.utcnow()} UTC</small>"
refresh_btn.on_click(on_refresh)  # register click callback

# --- NEW: auto-refresh loop (non-blocking) ---
async def _auto_loop():
    while auto_cb.value:
        refresh_dashboard()
        last_ref_html.value = f"<small>Last refresh: {pd.Timestamp.utcnow()} UTC</small>"
        await asyncio.sleep(2)  # non-blocking sleep in notebooks

def _on_auto_toggle(change):
    if change["name"] == "value":
        if change["new"]:
            asyncio.create_task(_auto_loop())  # start loop
        else:
            last_ref_html.value = "<small>Auto-refresh paused.</small>"
auto_cb.observe(_on_auto_toggle, names="value")  # traitlet change handler

# Initial render
refresh_dashboard()

ui = VBox([
    HTML("<h3>Order Simulator</h3>"),
    HBox([prod_dd, qty_sl, order_btn, refresh_btn, auto_cb]),
    last_ref_html,  # show last refresh time/status
    msg_out,
    HTML("<hr>"),
    kpi_html,
    HBox([
        VBox([HTML("<h4>Orders</h4>"), orders_out]),
        VBox([HTML("<h4>Products</h4>"), stock_out])
    ])
])
ui
